reference https://www.kaggle.com/gaoyuan19930220/xgboost

In [1]:
import numpy as np
import pandas as pd
import os
import sklearn
import xgboost as xgb
import operator
import optuna

from functools import partial
from matplotlib import pylab as plt
from sklearn import preprocessing
from sklearn.model_selection import cross_validate, StratifiedKFold


import warnings
warnings.filterwarnings('ignore')
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

/home/sano/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
count_itr = 0

In [3]:
def objective_optuna(X, y, trial):
    global count_itr
    print(count_itr, end=' ')
    count_itr += 1
    #最適化するパラメータを指定
    params = {
        'n_estimators': trial.suggest_int('n_estimators',1,1000),
        'gamma': trial.suggest_uniform('gamma',0,1),
        'reg_lambda': trial.suggest_uniform('lambda',0,2),
        'learning_rate': trial.suggest_uniform('learning_rate',0,0.3),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree',0,1),
        'min_child_weight': trial.suggest_uniform('min_child_weight',0,10),
    }
    
    #モデルを定義
    model = xgb.XGBClassifier(random_state=2,**params)
    
#     (train_X, test_X ,train_y, test_y) = train_test_split(X, y, test_size = 0.3, random_state = 2)
    
#     #3-foldクロスバリデーション
    kf = StratifiedKFold(n_splits=50, shuffle=True, random_state=2)
    scores = cross_validate(model, X=X, y=y, cv=kf, scoring='accuracy')
    
    return 1.0 - scores['test_score'].mean()

In [4]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [5]:
def preproceed(dataframe):
    # 欠損値の処理
    dataframe.replace({'Nan':pd.np.nan,'nan':pd.np.nan},inplace=True)
#     dataframe.fillna(dataframe.mean(numeric_only=True),inplace=True)
#     dataframe['Fare'].fillna(dataframe['Fare'].mean(),inplace=True)
#     dataframe['Embarked'].fillna(dataframe['Embarked'].mode().loc[0],inplace=True)
    
    #自分以外の家族の人数及び有無を追加
    dataframe['NumFamily'] = dataframe['SibSp'] + dataframe['Parch']
    dataframe['IsFamily'] = dataframe['NumFamily'] != 0

    #質的変数をダミー変数に
    dataframe['IsPclass1'] = dataframe['Pclass'] == 1
    dataframe['IsPclass2'] = dataframe['Pclass'] == 2
    dataframe['IsPclass2'] = dataframe['Pclass'] == 3
    dataframe['IsEmbarkedS'] = dataframe['Embarked'] == 'S'
    dataframe['IsEmbarkedC'] = dataframe['Embarked'] == 'C'
    dataframe['IsEmbarkedQ'] = dataframe['Embarked'] == 'Q'
    
    
    #質的変数を変換'
    dataframe['Sex'] = dataframe['Sex'].apply(lambda x: 1 if x == 'male' else 0)
#     dataframe['Embarked'] = dataframe['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2}).astype('int')


    #いらない列(Cabin,Name,PassengerId,Ticket)の削除
    dataframe.drop(['Cabin','Name','Ticket','Embarked','Pclass'],axis=1,inplace=True)

In [6]:
#csvデータの呼び出し場所
loadpath = "../input/"
#csvデータの保存場所
savepath = "../output/"

In [7]:
df = pd.read_csv(loadpath + "train.csv")
df_test = pd.read_csv(loadpath + "test.csv")

df.set_index('PassengerId',inplace=True)
df_test.set_index('PassengerId',inplace=True)


train_index = df.index
test_index = df_test.index

preproceed(df)
preproceed(df_test)

In [8]:
train = df.drop('Survived', axis=1)
labels = df.Survived
test = df_test
#データを分割
#(train_X, test_X ,train_y, test_y) = train_test_split(train_X, train_y, test_size = 0.3, random_state = 666)

In [9]:
train = train.astype('float')

In [ ]:
xgboost.callback.record_evaluation333

In [ ]:
f = partial(objective_optuna, train, labels)
study = optuna.create_study()
study.optimize(f, n_trials=10000)

0 

[I 2019-02-09 03:47:01,376] Finished a trial resulted in value: 0.1717401960784315. Current best value is 0.1717401960784315 with parameters: {'n_estimators': 434, 'gamma': 0.5726185523373011, 'lambda': 1.5709767643688664, 'learning_rate': 0.23645767229901932, 'max_depth': 13, 'subsample': 0.7413138238884882, 'colsample_bytree': 0.4620875878490819, 'min_child_weight': 3.426294474450702}.


1 

[I 2019-02-09 03:47:15,941] Finished a trial resulted in value: 0.17625000000000002. Current best value is 0.1717401960784315 with parameters: {'n_estimators': 434, 'gamma': 0.5726185523373011, 'lambda': 1.5709767643688664, 'learning_rate': 0.23645767229901932, 'max_depth': 13, 'subsample': 0.7413138238884882, 'colsample_bytree': 0.4620875878490819, 'min_child_weight': 3.426294474450702}.


2 

...

4438 

[I 2019-02-09 13:12:11,552] Finished a trial resulted in value: 0.1627777777777779. Current best value is 0.15277777777777768 with parameters: {'n_estimators': 55, 'gamma': 0.49314255136774965, 'lambda': 0.4995067747454911, 'learning_rate': 0.18632551931349686, 'max_depth': 14, 'subsample': 0.9993267322440277, 'colsample_bytree': 0.801340904839892, 'min_child_weight': 8.398175397205016}.


4439 

[I 2019-02-09 13:12:16,943] Finished a trial resulted in value: 0.18317810457516326. Current best value is 0.15277777777777768 with parameters: {'n_estimators': 55, 'gamma': 0.49314255136774965, 'lambda': 0.4995067747454911, 'learning_rate': 0.18632551931349686, 'max_depth': 14, 'subsample': 0.9993267322440277, 'colsample_bytree': 0.801340904839892, 'min_child_weight': 8.398175397205016}.


4440 

In [ ]:
# train a xgboost classifier
# params = {"num_class": 2}
# train_xgb = xgb.DMatrix(train, labels)
# test_xgb  = xgb.DMatrix(test)
# trainRound = 100
# gbm = xgb.train(params, train_xgb, trainRound)
# pred = gbm.predict(test_xgb)
# pred = pred.astype('int')

In [ ]:
study.best_params

In [ ]:
#最適化するパラメータを設定
    
#モデルの定義
model = xgb.XGBClassifier(random_state=2,**study.best_params)
model.fit(df.drop('Survived', axis=1), df.Survived,eval_metric='error')
    
kf = StratifiedKFold(n_splits=10, shuffle=True,)
scores = cross_validate(model, X=train, y=labels, cv=kf, scoring='accuracy')
# return 1.0 - scores['test_score'].mean()

In [ ]:
scores['test_score'].mean()

In [ ]:
model.save_model('xgbmodel')

In [ ]:
pred = model.predict(df_test)

In [ ]:
train.shape

In [ ]:
# create submission file
submission_result = pd.DataFrame({"Survived":pred},index=test.index)
submission_result.to_csv(savepath + 'submission_xgb.csv',index='False')

# 以下適当なコード

In [ ]:
#重要度のやつ


features = train.columns
# create a feature map
outfile = open('xgb.fmap', 'w')
i = 0
for feat in features:
    outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    i = i + 1
outfile.close()

# plot feature importance
importance = gbm.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))
df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()
plt.figure()
df.plot()
df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 20))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
plt.gcf().savefig('feature_importance_xgb.png')



In [31]:

#使うデータの前処理
train_xgb = xgb.DMatrix(df.drop('Survived', axis=1), df.Survived)
test_xgb  = xgb.DMatrix(df_test)
    
#最適化するパラメータを設定
params = {
     'n_estimators': 100,
     'gamma': 0,
     'reg_lambda': 1,
      #'max_depth': trial.suggest_int('max_depth', 2, 32)
}
    
#モデルの定義
model = xgb.XGBClassifier(random_state=2,**params)
model.fit(df.drop('Survived', axis=1), df.Survived)
    
# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
# scores = cross_validate(model, X=X, y=y, cv=kf, scoring='accuracy')
    
# return 1.0 - scores['test_score'].mean()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=2,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

AttributeError: 'XGBClassifier' object has no attribute 'get_fscore'